# Basic

In [26]:
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 원본 문서 데이터
docstore = InMemoryStore()
docs = [
    ("fake_id_1", Document(page_content="fake whole document 1")),
    ("fake_id_2", Document(page_content="fake whole document 2")),
]

# docstore에 원본 문서 저장
docstore.mset(docs)
docs

[('fake_id_1', Document(page_content='fake whole document 1')),
 ('fake_id_2', Document(page_content='fake whole document 2'))]

In [27]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import InMemoryStore
from langchain.retrievers import MultiVectorRetriever
from langchain.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# 서브 문서 데이터
sub_docs = [
    Document(
        page_content="A snippet from a larger document discussing cats.",
        metadata={"doc_id": "fake_id_1"},
    ),
    Document(
        page_content="A snippet from a larger document discussing discourse.",
        metadata={"doc_id": "fake_id_1"},
    ),
    Document(
        page_content="A snippet from a larger document discussing chocolate.",
        metadata={"doc_id": "fake_id_2"},
    ),
]

# 임베딩 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# FAISS 벡터 저장소 생성
vectorstore = FAISS.from_documents([Document(page_content="")], embeddings)

# vectorstore에 서브 문서 저장
vectorstore.add_documents(sub_docs)

['02c71d29-239e-4d5b-9248-9cff8c0e53ad',
 'a000cb15-4495-4101-9049-f23df74ac2e4',
 '6a094ae1-2a44-435f-ab08-de2afb04891a']

In [28]:
from langchain.storage import InMemoryByteStore

# MultiVectorRetriever 초기화
id_key = "doc_id"
multi_vector_retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key=id_key,
)

In [29]:
# vectorstore 검색 결과 보기
multi_vector_retriever.vectorstore.similarity_search("chocolate")[0]

Document(metadata={'doc_id': 'fake_id_2'}, page_content='A snippet from a larger document discussing chocolate.')

In [35]:
# docstore 검색 결과 보기 (vectorstore에 서브 문서들 중 유사도가 높은 서브 문서에 링크되어 있는 원본 문서 내용을 리턴)
multi_vector_retriever.invoke("chocolate")

[Document(page_content='fake whole document 2'),
 Document(page_content='fake whole document 1')]

# 요약

In [36]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import InMemoryStore
from langchain.retrievers import MultiVectorRetriever
from langchain.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# 문서 로드 및 분할
loader = TextLoader("../assets/ai-story.txt")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

# 임베딩 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# FAISS 벡터 저장소 생성
vectorstore = FAISS.from_documents(splits, embeddings)

In [2]:
docs

[Document(metadata={'source': '../assets/ai-story.txt'}, page_content='Scikit Learn\n\nScikit-learn은 Python 언어를 위한 또 다른 핵심 라이브러리로, 기계 학습의 다양한 알고리즘을 구현하기 위해 설계되었습니다. 이 라이브러리는 2007년 David Cournapeau에 의해 프로젝트가 시작되었으며, 그 후로 커뮤니티의 광범위한 기여를 받아 현재까지 발전해왔습니다. Scikit-learn은 분류, 회귀, 군집화, 차원 축소 등 다양한 기계 학습 작업을 지원하며, 사용하기 쉬운 API와 함께 제공되어 연구자와 개발자가 복잡한 데이터 과학 문제를 해결할 수 있도록 돕습니다.\n\n핵심 특징 중 하나는 다양한 기계 학습 모델을 일관된 인터페이스로 제공한다는 점입니다. 이는 사용자가 알고리즘 간에 쉽게 전환할 수 있게 하여, 최적의 모델을 찾는 과정을 단순화합니다. 또한, Scikit-learn은 사전 처리, 모델 선택, 평가 지표 등 기계 학습 파이프라인의 다른 단계에 필요한 도구들을 포함하고 있습니다. 이는 연구자와 개발자가 데이터를 더 효율적으로 처리하고, 모델의 성능을 정확히 평가할 수 있게 해줍니다.\n\nScikit-learn의 강점은 그의 범용성에 있습니다. 이는 다양한 산업과 연구 분야에서 사용될 수 있으며, 소규모 데이터셋부터 대규모 데이터셋까지 다양한 크기의 데이터를 처리할 수 있습니다. 또한, 오픈 소스 프로젝트로서, Scikit-learn은 지속적인 개선과 업데이트가 이루어지며, 사용자 커뮤니티로부터의 피드백과 기여를 받아 발전해 나갑니다.\n\n기계 학습 분야에 있어서, Scikit-learn은 특히 초보자에게 친화적인 학습 자원을 제공함으로써, 복잡한 이론과 알고리즘을 이해하는 데 중요한 역할을 합니다. 이러한 접근성과 범용성은 Scikit-learn을 기계 학습을 시작하는 이들에게 인기 있는 선택지로 만들었습니다.\n\nNLP\n\nNLP(자연어 처리)는 인간

In [37]:
# 요약 체인 생성
llm = ChatOllama(model="ko-gemma-2-9b-it.Q5_K_M:latest")
summarize_chain = (
    ChatPromptTemplate.from_template("다음 텍스트를 1-2문장으로 요약해주세요: {text}")
    | llm
    | StrOutputParser()
)

# 각 분할에 대한 요약 생성
summaries = []
for split in splits:
    summary = summarize_chain.invoke({"text": split.page_content})
    summaries.append(summary)

# MultiVectorRetriever 생성
store = InMemoryStore()
id_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    search_kwargs={"k": 5},
)

In [39]:
summaries

['Scikit-learn은 Python에서 기계 학습 알고리즘을 구현하기 위한 핵심 라이브러리로, 분류, 회귀, 군집화 등 다양한 작업을 지원합니다. 사용하기 쉬운 API를 통해 연구자와 개발자들이 데이터 과학 문제를 효율적으로 해결할 수 있도록 돕습니다.  \n\n\n',
 'Scikit-learn은 다양한 기계 학습 모델을 일관된 인터페이스로 제공하여 사용자들이 쉽게 알고리즘을 전환하고 최적의 모델을 찾을 수 있도록 돕는 강력한 도구입니다. 또한, 데이터 전처리부터 모델 평가까지 필요한 다양한 도구들을 포함하고 있으며, 오픈 소스로서 지속적인 발전을 이루고 있습니다. \n\n\n',
 'Scikit-learn은 기계 학습 초보자를 위한 친화적인 학습 자원을 제공하여 복잡한 이론과 알고리즘을 이해하는 데 도움을 주며, 이로 인해 기계 학습 시작자들에게 인기 있는 선택지가 되었습니다. \n\nNLP는 컴퓨터가 인간의 언어를 이해하고 처리하도록 하는 기술로, 기계 번역, 감정 분석, 음성 인식 등 다양한 분야에 활용됩니다. \n\n\n',
 '자연어 처리(NLP)의 핵심 과제는 인간 언어의 모호성과 다양성을 해결하는 데 있습니다. 최근 딥러닝, 특히 변환기 모델의 발전으로 인해 NLP 시스템은 문맥 분석 및 의미 이해 능력을 크게 향상시켜 자연어를 더 정확하게 처리하고 생성할 수 있게 되었습니다.',
 'NLP는 사람과 컴퓨터 간 소통 방식을 바꿔주었으며, 자연어 이해를 통한 정보 검색, 자동 번역, 감정 분석 등 다양한 분야에서 활용되고 있습니다. 앞으로도 NLP 연구는 인간 언어를 더 깊이 이해하고 인간-기계 상호작용을 향상시킬 것입니다. \n\n\n',
 'SciPy는 NumPy를 기반으로 한 파이썬 라이브러리로, 과학 계산을 위한 다양한 고급 수학 함수, 알고리즘, 유틸리티를 제공합니다. 최적화, 선형 대수, 적분 등 다양한 분야의 모듈을 통해 수학적 연산과 과학적 분석을 효율적으로 수행할 수 있습니다.',
 'SciPy는 연구자와 엔지니어가 복잡한 수

In [59]:
from langchain.schema import Document

# doc id 생성
doc_ids = [f"doc_{i}" for i in range(len(splits))]

# docstore에 원본 문서 저장
multi_vector_retriever.docstore.mset(list(zip(doc_ids, splits)))

summary_docs = []
for i, summary in enumerate(summaries):  # summaries 리스트의 각 요약과 인덱스에 대해 반복
    doc_id = doc_ids[i]  # 현재 인덱스에 해당하는 문서 ID 가져오기
    document = Document(page_content=summary, metadata={id_key: doc_id})  # Document 객체 생성
    summary_docs.append(document)  # 생성한 Document 객체를 리스트에 추가

# vectorstore에 요약 문서 저장
multi_vector_retriever.vectorstore.add_documents(summary_docs)

['154059c1-cadb-4223-a9ae-0cdd2a929a70',
 'adac1ab4-0c20-4f60-b711-f16c886139a3',
 '67bb9b0b-c21c-412e-a6d4-121af2dc0ec2',
 'a029cec0-b909-4813-a83c-f7df21adfa0f',
 '4ab45bda-1cb5-48e0-b136-40fba2726bb7',
 '5a6abfd4-b1ad-4556-9d61-417c4c896ff6',
 'ebc65dca-5572-41d7-9ad8-641c566a17d0',
 'db899e1f-3aa7-41ee-803e-67542167492d',
 '746969fd-750a-4878-a920-7b842db754af',
 '5b48556b-616e-438d-a503-11b57aabd003',
 '58d5c890-07d4-4736-becf-ec732abb324f',
 '31e688d0-795d-466b-a5ac-c499aed9f85d',
 '704c6790-6401-4110-9347-f5149b55dacb',
 '8a291253-7616-4cc0-9a0a-34d278f0ad7f',
 'd9596e61-50f6-4958-8d5f-9cb6349fa4cb',
 '24a2364c-14f6-45d9-8357-61a069d0a3a4',
 'e7b1b854-eeb6-46cc-984b-8d1f4b6f8a1d',
 '91235e00-ae6d-497a-8017-473c97ebd0e7',
 '08a6587c-ea4c-47c0-b223-61f2fec2e330',
 '7bb3fe82-1cea-4bff-87dd-eef201881f10']

In [65]:
# docstore 내부적으로 어떤 데이터 구조를 사용하는지 확인
print(vars(multi_vector_retriever.docstore))

{'store': {'doc_0': Document(metadata={'source': '../assets/ai-story.txt'}, page_content='Scikit Learn\n\nScikit-learn은 Python 언어를 위한 또 다른 핵심 라이브러리로, 기계 학습의 다양한 알고리즘을 구현하기 위해 설계되었습니다. 이 라이브러리는 2007년 David Cournapeau에 의해 프로젝트가 시작되었으며, 그 후로 커뮤니티의 광범위한 기여를 받아 현재까지 발전해왔습니다. Scikit-learn은 분류, 회귀, 군집화, 차원 축소 등 다양한 기계 학습 작업을 지원하며, 사용하기 쉬운 API와 함께 제공되어 연구자와 개발자가 복잡한 데이터 과학 문제를 해결할 수 있도록 돕습니다.'), 'doc_1': Document(metadata={'source': '../assets/ai-story.txt'}, page_content='핵심 특징 중 하나는 다양한 기계 학습 모델을 일관된 인터페이스로 제공한다는 점입니다. 이는 사용자가 알고리즘 간에 쉽게 전환할 수 있게 하여, 최적의 모델을 찾는 과정을 단순화합니다. 또한, Scikit-learn은 사전 처리, 모델 선택, 평가 지표 등 기계 학습 파이프라인의 다른 단계에 필요한 도구들을 포함하고 있습니다. 이는 연구자와 개발자가 데이터를 더 효율적으로 처리하고, 모델의 성능을 정확히 평가할 수 있게 해줍니다.\n\nScikit-learn의 강점은 그의 범용성에 있습니다. 이는 다양한 산업과 연구 분야에서 사용될 수 있으며, 소규모 데이터셋부터 대규모 데이터셋까지 다양한 크기의 데이터를 처리할 수 있습니다. 또한, 오픈 소스 프로젝트로서, Scikit-learn은 지속적인 개선과 업데이트가 이루어지며, 사용자 커뮤니티로부터의 피드백과 기여를 받아 발전해 나갑니다.'), 'doc_2': Document(metadata={'source': '../assets/ai-story.txt'}, page_content='기계 학

In [61]:
# vectorstore 검색 결과 보기
multi_vector_retriever.vectorstore.similarity_search("Word2Vec")[0]

Document(metadata={'doc_id': 'doc_18'}, page_content='Word2Vec은 CBOW와 Skip-Gram 두 가지 모델을 통해 단어를 벡터로 표현하는 기술입니다. 이 벡터 표현은 단어 유사도 측정, 문장 벡터화, 기계 번역, 감정 분석 등 다양한 자연어 처리 작업에 활용되며, 벡터 연산을 통해 단어 간 의미적 관계를 분석할 수 있습니다.')

In [63]:
# docstore 검색 결과 보기 (vectorstore에 서브 문서들 중 유사도가 가장 뛰어난 서브 문서에 링크되어 있는 원본 문서 내용을 리턴)
multi_vector_retriever.invoke("Word2Vec")

[Document(metadata={'source': '../assets/ai-story.txt'}, page_content='Word2Vec은 크게 두 가지 모델 아키텍처로 구성됩니다: Continuous Bag-of-Words (CBOW)와 Skip-Gram입니다. CBOW 모델은 주변 단어(맥락)를 기반으로 특정 단어를 예측하는 반면, Skip-Gram 모델은 하나의 단어로부터 주변 단어들을 예측합니다. 두 모델 모두 딥러닝이 아닌, 단순화된 신경망 구조를 사용하여 대규모 텍스트 데이터에서 학습할 수 있으며, 매우 효율적입니다.\n\nWord2Vec의 벡터 표현은 다양한 NLP 작업에 활용될 수 있습니다. 예를 들어, 단어의 유사도 측정, 문장이나 문서의 벡터 표현 생성, 기계 번역, 감정 분석 등이 있습니다. 또한, 벡터 연산을 통해 단어 간의 의미적 관계를 추론하는 것이 가능해집니다. 예를 들어, "king" - "man" + "woman"과 같은 벡터 연산을 수행하면, 결과적으로 "queen"과 유사한 벡터를 가진 단어를 찾을 수 있습니다.')]